# Demystifying Machine Learning 
## Using  Local Interpretable Model-Agnostic Explanations (LIME)

by [Alejandro Correa Bahnsen](http://www.albahnsen.com/)

Machine learning models are often dismissed on the grounds of lack of interpretability. There is a popular story about modern algorithms that goes as follows: Simple linear statistical models such as logistic regression yield to interpretable models, on the other hand, advanced models such as random forest or deep neural networks are black boxes. Meaning that it is nearly impossible to understand how a model is making a prediction. 

<center>
<img src="figures/black-box.PNG" style="width: 300px;" align="middle">
</center>

The infosec industry is very used to rules, blacklisting, fingerprints and indicators of compromise so explaining why an alert went off is easy and natural. In contrast, machine learning models are able to identify complex nonlinear patterns in large data sets, extrapolate answers and make predictions based on non-trivial compositions that make it nearly impossible to get a feeling for its inner workings.

Then, if I can use simple expert rules, or regression/decision tree models that are interpretable, why should I use non-interpretable models? Well, because complex machine learning models are much more powerful. 

For example, in the next figure I’m showing the implicit tradeoff between model performance, measured in accuracy, and model complexity, measured in number of parameters. This is built using the common [MINST handwritten digits dataset](http://yann.lecun.com/exdb/mnist/), where the objective is to identify which number was written by looking only at the pixel values of each image.


<center>
<img src="figures/complexity.png" style="width: 600px;" align="middle">
</center>
* [Notebook to create the above plot](http://nbviewer.jupyter.org/github/albahnsen/Talk_Demystifying_Machine_Learning/blob/master/figures/Create_Model_Complexity_Figure.ipynb)

As we can see, as a general rule, the more complex the model the higher its performance. In this case, using a simple interpretable decision tree yields an accuracy of 86%, on the other hand a convolutional neural network has an accuracy of 99.3%, however, it must train more than a million parameters, compared with only 1,700 of the decision tree. 

Big efforts are being invested in finding ways to explain the output of machine learning models. Just over a couple months ago, a novel method [LIME](http://arxiv.org/abs/1602.04938) was presented during the 2016 ACM's Conference on Knowledge Discovery and Data Mining. LIME stands for Local Interpretable Model-agnostic Explanations, and its objective is to explain the result from any classifier so that a human can understand individual predictions.


<center>
<img src="https://d3ansictanv2wj.cloudfront.net/figure1-a9533a3fb9bb9ace6ee96b4cdc9b6bcb.jpg" style="width: 800px;" align="middle">
</center>
* [Source](https://www.oreilly.com/learning/introduction-to-local-interpretable-model-agnostic-explanations-lime)


The LIME algorithm approximates the underlying model with an interpretable one. This is done by learning on perturbations of the original example, and training a sparse linear model  in the nearest neighborhood around the target instance. Let's see how that is done with this example. 

<center>
<img src="https://raw.githubusercontent.com/marcotcr/lime/master/doc/images/lime.png" style="width: 500px;" align="middle">
</center>
* [Source](https://github.com/marcotcr/lime)


In the figure, the black-box complex decision function is represented by the blue/pink background. As expected there is no easy way to generalize this non-linear model. However, if we look locally, we can extract linear explanations. LIME, first select the nearest neighbors to the prediction that we want to explain. These nearest neighbours are represented by the big crosses and circles. Afterwards, we can fit an explainable linear model using only the selected instances, and use the coefficients of the linear model to explain a particular prediction.

I know this sounds way to confusing, but allow me to show an example of using LIME to explain the prediction of the [Swordphish](http://blog.easysol.net/the-technical-side-of-phishing/) (Phishing URLs classifier) model.

## Using LIME to explain Phishing URLs predictions

We recently made accessible to the public our machine learning based Swordphish phishing URL prediction system. Swordphish is a REST API backed by a set of extremely fast machine learning classifiers designed to predict with a high-degree of confidence if a URL or domain is likely to be associated with phishing or malware command and control (C&C). Swordphish estimates approximately 50 features based on analyzing the structure of the URL, for example by estimating Kullback-Leibler Divergence between the normalized character frequency of the English language and the URL. Other features include the number of @ and - symbols, the number of top-level domains in the URL, whether the URL is an IP address, the URL length and the number of suspicious words in the URL. 

Afterwards, Swordphish uses a Random Forest classifier that was trained with a total of 1.3 million parameters, and predicts the probability of an URL being a phishing site. As I pointed out above, understanding the prediction of such a complex machine learning model is nearly impossible. However, using LIME we can approximate the reasons of how Swordphish is making individual predictions. 